In [57]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [41]:
# Constantes
HEIGHT = 224
WIDTH = 224
CHANNELS = 3
NUM_EPOCHS = 5
NUM_CLASS = 2
BATCH_SIZE = 32

In [14]:
# Load MobileNetV2 from TensorFlow Hub
base_model = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5", trainable=True)

In [42]:
# Cargar el archivo CSV
df = pd.read_csv('./data/working-table.csv')

# Obtener las rutas de las imágenes y las etiquetas correspondientes
#image_paths = ['./data/procesadas/' + name for name in df['name']]
image_paths = df['name']
labels = df['category']
df

,name,category
0,./data/procesadas/can/no_acceptable/imagen_1.jpg,no_aceptable
1,./data/procesadas/can/no_acceptable/imagen_2.jpg,no_aceptable
2,./data/procesadas/can/no_acceptable/imagen_3.jpg,no_aceptable
3,./data/procesadas/can/no_acceptable/imagen_4.jpg,no_aceptable
4,./data/procesadas/can/no_acceptable/imagen_5.jpg,no_aceptable
...,...,...
195,./data/procesadas/plastic/acceptable/imagen_46...,aceptable
196,./data/procesadas/plastic/acceptable/imagen_47...,aceptable
197,./data/procesadas/plastic/acceptable/imagen_48...,aceptable
198,./data/procesadas/plastic/acceptable/imagen_49...,aceptable


In [43]:
# Process images and labels
processed_images = []
processed_labels = []

for image_path, label in zip(df['name'], df['category']):
    try:
        try:
            # Load the image
            image = load_img(image_path, target_size=(HEIGHT, WIDTH))
            # Convert the image to a NumPy array
            image_array = img_to_array(image)
            image_array = preprocess_input(image_array)  # Apply MobileNetV2-specific preprocessing
            # Append the preprocessed image and label to the corresponding lists
            processed_images.append(image_array)
            
            # Convert label to its corresponding numerical value
            if "no_aceptable" in label:
                processed_labels.append(0)
            else:
                processed_labels.append(1)
        except Exception as ex:
            print(ex)
            continue
    except Exception as ex:
        print(ex)
        continue

# Convert lists to NumPy arrays
processed_images = np.array(processed_images)
processed_labels = np.array(processed_labels)

cannot identify image file <_io.BytesIO object at 0x167a2a070>
cannot identify image file <_io.BytesIO object at 0x167ad2cf0>
cannot identify image file <_io.BytesIO object at 0x167b132e0>


In [17]:
#processed_images
processed_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [44]:
# Build fine-tuned model
model = models.Sequential([
    base_model,
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation='softmax')  # Update to NUM_CLASS
])

model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
              metrics=['accuracy'])

In [45]:
# Dividir los datos en conjuntos de entrenamiento y prueba

datos_entrenamiento, datos_prueba, etiquetas_entrenamiento, etiquetas_prueba = train_test_split(
    processed_images, processed_labels, test_size=0.2, random_state=42)

In [46]:
# Fine-tuning
history = model.fit(datos_entrenamiento, etiquetas_entrenamiento,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    shuffle=True)

Epoch 1/5
5/5 [==============================] - 30s 3s/step - loss: 1.1374 - accuracy: 0.5287
Epoch 2/5
5/5 [==============================] - 15s 3s/step - loss: 0.2859 - accuracy: 0.9363
Epoch 3/5
5/5 [==============================] - 15s 3s/step - loss: 0.1903 - accuracy: 0.9745
Epoch 4/5
5/5 [==============================] - 15s 3s/step - loss: 0.1662 - accuracy: 0.9936
Epoch 5/5
5/5 [==============================] - 15s 3s/step - loss: 0.1465 - accuracy: 0.9936


In [47]:
# Evaluar el modelo
pérdida, precisión = model.evaluate(datos_prueba, etiquetas_prueba)
print('Pérdida:', pérdida)
print('Precisión:', precisión)

2/2 [==============================] - 1s 134ms/step - loss: 0.1676 - accuracy: 0.9750
Pérdida: 0.16762223839759827
Precisión: 0.9750000238418579


In [56]:
# Generate predictions on the training data
y_pred_entrenamiento = model.predict(datos_entrenamiento)

# Calculate predicted probabilities for the positive class
y_pred_entrenamiento_positive_class = y_pred_entrenamiento[:, 1]

# Convert predicted probabilities to binary class labels
y_pred_entrenamiento_binary = (y_pred_entrenamiento_positive_class > 0.5).astype(int)

# Calculate F1-Score for training data
f1_train = f1_score(etiquetas_entrenamiento, y_pred_entrenamiento_binary)

print('F1-Score on Training Data:', f1_train)



5/5 [==============================] - 3s 483ms/step
F1-Score on Training Data: 1.0


In [54]:
# Probar el modelo
from tensorflow.keras.preprocessing import image as keras_image
from PIL import Image

# Ruta de la imagen que deseas utilizar para la predicción
image_path = 'data/procesadas/can/no_acceptable/imagen_2.jpg'

# Cargar la imagen
image = Image.open(image_path)


# Redimensionar la imagen a (HEIGHT, WIDTH)
image = image.resize((WIDTH, HEIGHT))

# Convertir la imagen a un array numpy y normalizar los valores de píxeles
image_array = keras_image.img_to_array(image)
image_array = image_array / 255.0

# Agregar una dimensión extra para representar el batch (el modelo espera un lote de imágenes)
input_data = np.expand_dims(image_array, axis=0)

predictions = model.predict(input_data)

# Format prediction probabilities to display 2 decimal places
formatted_predictions = [format(prob, ".2f") for prob in predictions[0]]

# Print the formatted predictions
print("Formatted Predictions:", formatted_predictions)

1/1 [==============================] - 0s 42ms/step
Formatted Predictions: ['0.99', '0.01']


In [55]:
model.save('cnn_model_transfer_learning.h5')

/Users/jbaltod1/Documents/UCenfotec/RecyCam-Model/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
